In [1]:
# ### File to automate the data parsing of the output files (excel files of energies) and create the required graphs 
# and input-output csv file for ML network input

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from numpy import trapz
import numpy as np

M = 100 #Number of models

### Read job output files and creat the figures for energies and RF-U

for ii in range(1,M+1):
    data = pd.read_csv("Dynamic_Job-%d_Energies_RF_U.csv" %(ii))
    t = data["Time_steps"][:]
    AE = data["Artificial_Energy"][:]
    IE = data["Internal_Energy"][:]
    KE = data["Kinetic_Energy"][:]
    TE = data["Total_En"][:]
    U3 = data["Disp_Im"][:]
    RF = data["Contact_Normal_Force"][:]
    plt.plot(t, AE, color='b', label='Artificial Strain Energy')
    plt.plot(t, IE, color='g', label='Internal Energy')
    plt.plot(t, KE, color='r', label='Kinetic Energy')
    plt.plot(t, TE, color='k', label='Total Energy')
    plt.legend()
    plt.title("Energies - time")
    plt.xlabel('Time (s)')
    plt.ylabel('Energy (KJ)')
    plt.savefig('Job-%d_Energies.png' %(ii))
    plt.close()    # close the figure window
    plt.plot(U3,RF, color='violet', label='Reaction force of CB')
    plt.legend()
    plt.title("Reaction force - displacement")
    plt.xlabel('Displacement of impactor (mm)')
    plt.ylabel('Reaction force (N)')
    plt.savefig('Job-%d_RF_Disp.png' %(ii))
    plt.close()    # close the figure window

In [3]:
### Calculate the peak impact force, energy absorbed, total deformation and mean impact force in every job file

# Two new dictionaries to store the peak impact force and energy abosorbed for every job
max_value = []
area = []
Deforamtion = []
Mean_force = []
CB_weight = []

for jj in range(1,M+1):
    data = pd.read_csv("Dynamic_Job-%d_Energies_RF_U.csv" %(jj))
    t = data["Time_steps"][:]
    IE = data["Internal_Energy"][:]
    RF = data["Contact_Normal_Force"][:]
    AE = data["Artificial_Energy"][:]
    U3 = data["Disp_Im"][:]
    mass_CB = data["CB_mass"][:]
    
    MV = round(RF[:50].max()/1000,5)
    max_value.append(MV)

    EA = round((IE[200]-AE[200])/1000000,5)
    area.append(EA)
    
    d = round(U3.max(),4)-10
    Deforamtion.append(d)
    
    P_mean = round((EA*1000)/d,5)
    Mean_force.append(P_mean)
    
    mass = mass_CB.max()*1000000
    CB_weight.append(mass)

In [9]:
# read the previously generated file for model and it's parameters
df = pd.read_csv('model_database_dynamic_impact_sims.csv')

# Add columns in right to add the peak impact force and energy absorbed
df["Peak_Impact_Force_KN"] = max_value
df['Energy_Absorbed_KJ'] = area
df['Deformation_CB_mm'] = Deforamtion
df['Mean_impact_force_KN']= Mean_force
df['Mass_CB_g'] = CB_weight

In [11]:
print(isinstance(df, dict))
print(df.keys())

False
Index(['Model', 'Width', 'Height', 'Thickness', 'Length', 'Velocity',
       'Mass_Im', 'Dynamic_Job', 'Time_req_s', 'Peak_Impact_Force_KN',
       'Energy_Absorbed_KJ', 'Deformation_CB_mm', 'Mean_impact_force_KN',
       'Mass_CB_g'],
      dtype='object')


In [12]:
# Save the new file in excel file with new name 'Model_Database'
df.to_excel('MODELS_DATABASE.xlsx', index=False)
df.to_csv('MODELS_DATABASE.csv', index=False)